<center><h2> Create employees JSON document </h2></center>

### Load libraries 

In [66]:
import json 
import uuid 
import random
import csv
import pandas as pd 
import numpy as np

### variables for JSON

In [61]:
# distribution for value is menager. 20% for being a menager.
menager_distribution = [0.2,0.8]


In [72]:
# load file with polish names. 
with open("data_for_JSON/names.csv", newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    names_list = list(reader)
    del names_list[0]

In [168]:
#Exceptions of typical names. 
name_men_exceptions = ["Jarema", "Barnaba", "Kuba", "Kosma", "Bonawentura", "Dyzma"]
name_women_excepions = ["Nicole"]

In [195]:
#load data from GUS
data_surnames_men = pd.read_csv("data_for_JSON/surnames_gus_men.csv", skiprows=1, names = ["surname","number"])
data_surnames_women = pd.read_csv("data_for_JSON/surnames_gus_women.csv", skiprows=1, names = ["surname","number"])


In [202]:
#Create column with perc value
data_surnames_men = create_perc_col(data_surnames_men)
data_surnames_women = create_perc_col(data_surnames_women)

In [294]:
data_age_men = pd.read_excel("data_for_JSON/age_bael.xlsx", sheet_name = "men")
data_age_women = pd.read_excel("data_for_JSON/age_bael.xlsx", sheet_name = "women")

In [295]:
data_age_men = create_perc_col(data_age_men)
data_age_women = create_perc_col(data_age_women)

In [305]:
data_age_men["age"] = data_age_men["age"].str.strip().str.replace("65  lat i więcej", "65-72")
data_age_women["age"] = data_age_women["age"].str.strip().str.replace("65  lat i więcej", "65-72")

In [321]:
random_value = data_age_men.sample(1, weights = 'perc').age.to_string(index = False).strip()
start_age = int(random_value[:2])
end_age = int(random_value[-2:])
employee_age = random.randint(start_age, end_age)

In [430]:
salary_worker_distribution = [0.5,0.35, 0.1, 0.05]
salary_menager_distribution = [0.25, 0.5, 0.15, 0.06, 0.04]
minimum_wage = 2600

[2850]

### Generate values for employees JSON

In [253]:
id = uuid.uuid4()
is_menager = random.choices([True, False], menager_distribution)[0]
name = random.choice(names_list)[0]
#Generate gender - based on name! 

if name in name_men_exceptions:
    #observation is men with name ending with a
    gender = 1
elif name in name_women_excepions:
    gender: 1
elif name[-1] is 'a':
    #observation is woman
    gender = 0
else:
    #observation is man
    gender = 1

if gender is 1:
    surname = get_men_surname()
    age = create_emplo_age(data_age_men)
else:
    surname = get_women_surname()
    age = create_emplo_age(data_age_women)
    
if is_menager: 
    salary = 
else:
    salary = random.choices([minimum_wage, create_interval_wage(50, 300),create_interval_wage(350, 600), create_interval_wage(650, 1000) ], salary_worker_distribution)

ja


### Functions
TO DO: add functions note book (or python file) and create import

In [251]:
def get_men_surname():
    """
    docstring
    """
    return data_surnames_men.sample(1, weights = 'perc').surname.to_string(index = False).strip().capitalize()

In [250]:
def get_women_surname():
    """
    docstring
    """
    return data_surnames_women.sample(1, weights = 'perc').surname.to_string(index = False).strip().capitalize()

In [270]:
def create_perc_col(data_frame, number_col: str = "number", ndigits: int = 2):
    """
    docstring
    """
    data_frame["perc"] = round(data_frame[number_col]/data_frame[number_col].sum(), ndigits)
    return data_frame

In [325]:
def create_emplo_age(age_data_set):
    """
        docstring
    """
    random_value = age_data_set.sample(1, weights = 'perc').age.to_string(index = False).strip()
    start_age = int(random_value[:2])
    end_age = int(random_value[-2:])
    return random.randint(start_age, end_age)

In [362]:
def create_interval_wage(min_add:int, max_add: int, step: int = 50, minimum_wage: int = 2600):
    return random.randrange(minimum_wage + min_add, minimum_wage + max_add, step)